In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [38]:
filename='data_cardiovascular_risk.csv'

In [39]:
df_nan = pd.read_csv('data_cardiovascular_risk.csv')

In [40]:
df = df_nan.dropna()
print(df)

        id  age  education sex is_smoking  cigsPerDay  BPMeds  \
1        1   36        4.0   M         NO         0.0     0.0   
2        2   46        1.0   F        YES        10.0     0.0   
3        3   50        1.0   M        YES        20.0     0.0   
4        4   64        1.0   F        YES        30.0     0.0   
5        5   61        3.0   F         NO         0.0     0.0   
...    ...  ...        ...  ..        ...         ...     ...   
3384  3384   61        1.0   F         NO         0.0     0.0   
3385  3385   60        1.0   F         NO         0.0     0.0   
3386  3386   46        1.0   F         NO         0.0     0.0   
3387  3387   44        3.0   M        YES         3.0     0.0   
3389  3389   54        3.0   F         NO         0.0     0.0   

      prevalentStroke  prevalentHyp  diabetes  totChol  sysBP  diaBP    BMI  \
1                   0             1         0    212.0  168.0   98.0  29.77   
2                   0             0         0    250.0  116.0

In [41]:
#df['is_smoking']=df["is_smoking"].map({"YES":1,"NO":0})
df['sex']=df["sex"].map({"M":1,"F":0})
df=df.drop(["id"],axis=1)
data_features = df.drop(['is_smoking'], axis=1)
print(data_features.head())

   age  education  sex  cigsPerDay  BPMeds  prevalentStroke  prevalentHyp  \
1   36        4.0    1         0.0     0.0                0             1   
2   46        1.0    0        10.0     0.0                0             0   
3   50        1.0    1        20.0     0.0                0             1   
4   64        1.0    0        30.0     0.0                0             0   
5   61        3.0    0         0.0     0.0                0             1   

   diabetes  totChol  sysBP  diaBP    BMI  heartRate  glucose  TenYearCHD  
1         0    212.0  168.0   98.0  29.77       72.0     75.0           0  
2         0    250.0  116.0   71.0  20.35       88.0     94.0           0  
3         0    233.0  158.0   88.0  28.26       68.0     94.0           1  
4         0    241.0  136.5   85.0  26.42       70.0     77.0           0  
5         0    272.0  182.0  121.0  32.80       85.0     65.0           1  


C:\Users\Hi\AppData\Local\Temp\ipykernel_7304\3466657744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex']=df["sex"].map({"M":1,"F":0})


In [42]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import pickle

X = data_features.drop(['TenYearCHD'], axis=1)
Y = data_features['TenYearCHD']

In [43]:
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
# The numbers before SMOTE
num_before = dict(Counter(Y))

# Perform SMOTE
# Define Pipeline
over=SMOTE(sampling_strategy=0.8)
under=RandomUnderSampler(sampling_strategy=0.8)
steps=[("o",over),("u",under)]
pipeline = Pipeline(steps=steps)

# Transform the dataset
X_smote,y_smote=pipeline.fit_resample(X,Y)
# The numbers after SMOTE
num_after=dict(Counter(y_smote))

In [44]:
# new dataset
new_data = pd.concat([pd.DataFrame(X_smote), pd.DataFrame(y_smote)], axis=1)
new_data.columns = ['age','education' ,'sex', 'cigsPerDay', 'BPMeds', 'prevalentStroke', 'prevalentHyp',
'diabetes', 'totChol', 'sysBP', 'diaBP','BMI', 'heartRate', 'glucose', 'TenYearCHD']
new_data.head()

,age,education,sex,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,61,2.0,0,0.0,0.0,0,0,0,219.0,120.0,72.5,22.35,76.0,92.0,0
1,36,3.0,0,20.0,0.0,0,0,0,159.0,121.5,73.0,20.41,72.0,75.0,0
2,38,2.0,0,7.0,0.0,0,0,0,160.0,95.0,65.0,21.99,72.0,77.0,0
3,48,2.0,1,25.0,0.0,0,1,0,249.0,132.0,95.0,29.79,90.0,53.0,0
4,62,1.0,1,5.0,0.0,0,1,0,243.0,157.0,96.0,28.83,75.0,71.0,0


In [45]:
X_new = new_data.iloc[:, 0:14]
Y_new= new_data.iloc[:,-1]
X_new.head()

,age,education,sex,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
0,61,2.0,0,0.0,0.0,0,0,0,219.0,120.0,72.5,22.35,76.0,92.0
1,36,3.0,0,20.0,0.0,0,0,0,159.0,121.5,73.0,20.41,72.0,75.0
2,38,2.0,0,7.0,0.0,0,0,0,160.0,95.0,65.0,21.99,72.0,77.0
3,48,2.0,1,25.0,0.0,0,1,0,249.0,132.0,95.0,29.79,90.0,53.0
4,62,1.0,1,5.0,0.0,0,1,0,243.0,157.0,96.0,28.83,75.0,71.0


In [46]:
X_train, X_test,y_train, y_test = train_test_split(X_new,Y_new,test_size=0.4,random_state=40)

In [47]:
#Chuẩn hóa dữ liệu
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_train=pd.DataFrame(X_train_scaled)

X_test_scaled=scaler.transform(X_test)
X_test=pd.DataFrame(X_test_scaled)

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, 30],  # Tăng giới hạn độ sâu
    'min_samples_split': [5, 10],  # Tăng ngưỡng tối thiểu cho phân chia nút
    'min_samples_leaf': [2, 4],  # Tăng ngưỡng tối thiểu cho lá
    'max_features': ['auto', 'sqrt']
}
# Instantiate the RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)

lr = GridSearchCV(rf_classifier, param_grid, cv=5, n_jobs=-1)
model =lr.fit(X_train,y_train)

C:\Users\Hi\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [49]:
#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import ExtraTreesClassifier
#lr=ExtraTreesClassifier()

#model =lr.fit(X_train,y_train)

In [50]:
# Lưu mô hình vào tệp model.pkl
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

In [51]:
prediction = model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, prediction)

0.8489932885906041

In [52]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, prediction)
print(cm)

[[880  94]
 [176 638]]


In [53]:
from sklearn.metrics import classification_report
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.83      0.90      0.87       974
           1       0.87      0.78      0.83       814

    accuracy                           0.85      1788
   macro avg       0.85      0.84      0.85      1788
weighted avg       0.85      0.85      0.85      1788



In [61]:
input_data = [50,1,0,0,0,0,1,0,240,163,105,31.37,89,75]
features = np.array(input_data).reshape(1, -1)
prediction = model.predict(features)
prediction

array([1], dtype=int64)